# Investment Analyst Assistant Retreival Notebook

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and limitations under the License.

Copyright 2024 Amazon Web Services, Inc.

##### This notebook allows you to generate metadata forom the question. This metadata will be used to retreived specific chunks from the Opensearch Index which is then sent to the LLM to generate answer for the specific question. This notebook is configured to use "anthropic.claude" LLMs.


**Question & Answer Pipeline**: 
![iaa_arch.png](../images/generation.png)
<!-- <center>
<img src="../src/generation.png" alt="Investment Analyst Assistant Architecture" width="400"/>
</center> -->

1. A user provides a query.
2. The Query Expansion/Enrichment module uses prompt engineering techniques to expand and enrich the query.
3. The expanded query embeddings are generated using the Titan Text Embedding Model.
4. The Retrieval component retrieves relevant chunks from the OpenSearch instance based on the query embeddings.
5. The retrieved chunks are passed to the Answer Generation module, which involves prompt engineering and interaction with a language model (OpenAI or Claude) to generate the final answer.

### STEP 0:  Reset And Install missing Packages

NOTE: Warnings and in some case, version errors can be ignored for package installation. Those are due to version updates. Only change versions if necessary.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install requests_toolbelt --quiet

#### Adding Project Directory to Path

In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust this path as needed
sys.path.append(project_root)

### STEP 1: Import Necessary Modules

In [2]:
from libraries.iaa.experiments.utils_exp import get_titan_text_embedding, results_fusion
from libraries.iaa.query_transformation.ExtractQueryMetadata import ExtractQueryMetadata
from libraries.iaa.reranker.SearchRanker import SearchRanker
from libraries.iaa.retrieval.OpenSearchRetrieval import OpenSearchRetrieval
import time
from dataclasses import dataclass, field
from libraries.iaa.query_transformation.QueryMetaExtractor import QueryMetaExtractor
import json
import boto3
import re
from bs4 import BeautifulSoup
from typing import List
import csv
import libraries.iaa.configs as configs

### STEP 2: Notebook Configuration

In [3]:
exp_config = {
    'type_rets': ['fusion'], #fusion (text and embb) is selected for this workshop because it generated the best results
    'opensearch_host': configs.OPEN_SEARCH_HOST, # copy the url created in the index creation notebook
    'generation_llm_model': 'anthropic.claude-3-haiku-20240307-v1:0',
    'index_name_embb': 'expt_index', #name of the index created in the index creation notebook
    'index_name_text': 'expt_index', #name of the index created in the index creation notebook
    'top_k_ranking': 20,
    'top_k_retrieval': 30,
    'emb_name': 'vector_field',
    'region': configs.REGION
            }

#### Input Question

In [4]:
question = "What was the revenue for 3M in 2022?"

### STEP 3 : Rephrase And Answer Generation

##### Initialize Bedrock Client

In [5]:
bedrock_client = boto3.client("bedrock-runtime", region_name=exp_config['region'])

#### Prompt to Extract Metadata from the Quesion

In [6]:
##Prompt To Extract Time Related Keyword

PROMPT_METADATA_GENERATION_time = """
\n\nHuman:
You a financial editer that looks at a user questions and rephrases it accurately for better search and retrieval tasks.

Financial question related to yearly and Quarterly financial Reports: {query} \n
Current year is {most_recent_year}
Current quarter is {most_recent_quarter}
<task>
Given a user question, identify the following metadata a list of time-related keywords based on instruction below
1. time_keyword_type: identifies what type of time range user is requesting for - range of years, range of quarters, specific years or specific quarters, none
2. time_keywords: these keywords expand the year or quarter period if time_keyword_type is "range of periods" else it will be formatted version of year in YYYY format or quarter in Q'YY format.
</task>

<instruction>
1. Identify whether the user is asking for a date range or specific set of years or quarters. If there is no year or quarter mention leave time_keyword blank
2. If the user is requesting for specific year or years return year in YYYY format.
3. If the user is requesting for specific quarter or quarters return quarter in Q'YY format. Example Q2'24, Q1'23
4. If the user is requesting for documents in a specific range of time between two period, fill the year or quarter information between the time ranges.
5. If the user is requesting for last N years, count backward from current year 2024
6. If the user is requesting for last N quarters, count backward from current quarter and year Q1 2024
<instruction>

<examples>
what was Google's net profit?
time_keyword_type: none
time_keywords: none
explanation: no quarter or year mentioned

What was Amazon's total sales in 2022?
time_keyword_type: specific_year
time_keywords: 2022

What was Apple's revenue in 2019 compared to 2018?
time_keyword_type: specific_year
time_keywords: 2018, 2019
explanation: the user is requesting to compare 2 different years

Which of Disney's business segments had the highest growth in sales in Q4 F2023?
time_keyword_type: specific_quarter
time_keywords: Q4 2023

How did Netflix's quarterly spending on research change as a percentage of quarterly revenue change between Q2 2019 and Q4 2019?
time_keyword_type: range_quarter
time_keywords: Q2 2019, Q3 2019, Q4 2019
explanation: the quarters between Q2 2019 and Q4 2019 are Q2 2019, Q3 2019 and Q4 2019

What was Spotify's growth in the last 5 quarters?
time_keyword_type: range_quarter
time_keywords: Q4 2023, Q3 2023, Q2 2023, Q1 2023, Q4 2024
explanation: Since current quarter is Q1 2024, the last 3 quarters are Q4 2023, Q3 2023, Q2 2023

In their 10-K filings, has Norweigean Cruise mentioned any negative environmental or weather-related impacts to their business in the last four years?
time_keyword_type: range_year
time_keywords: 2020, 2021, 2022, 2023
explanation: Since the current year is 2024, the last four years are 2020, 2021, 2022 and 2023.
</examples>


Return a JSON object with the following fields:
   - 'time_keyword_type': a list of time-related keywords
   - 'time_keywords': a list of technical keywords
   - 'explanation': explanation of you chose a certain time_keyword type and time keyword

\n\nAssistant:The metadata for the user question {query}:

"""


In [7]:
##Prompt To Extract technical Keywords

PROMPT_METADATA_TECHNICAL_KWD = """

Human: imagine you are a financial analyst looking to answer the question {query} in 10k/10q documents.

What are some of the keywords you would use for searching the documents based on the question?
<instruction>
1. Do not include company names, document names and timelines
2. Generate 5-6 important list of comma separated keywords within a single <keywords></keywords> tag.
3. Focus more on what sections of the document you would look at and add that to the keyword
4. Do not add keywords that are not part of the question
</instruction>


Assistant:

"""

In [8]:
##Prompt To Extract Metadata
PROMPT_METADATA_AND_QUERY_ReWr = """
\n\nHuman:
You a financial editer that looks at a user questions and rephrases it accurately for better search and retrieval tasks.

Financial question related to yearly and Quarterly financial Reports: {query} \n
<task>
Given a user question, identify the following metadata
   - 'technical_keywords': a list of relevant keywords from question
   - 'company_keywords': a list of company names
   - 'rephrased_question': the full rephrased question string
</task>

<time_keywords>
{time_kwds}
</time_keywords>

<technical_keywords>
1. Generate a comprehensive list of all possible keywordsthat are relevant based on sections you would typically find in a financial document.
2. Include different alternatives to the keywords, be imaginative.
3. Remove the company name and document name from keyword list.
</technical_keywords>

<company_keywords>
Generate a list of company names that are mentioned in the question.
</company_keywords>

<rephrased_question>
1. Generate the keywords and rephrase the question to make it very clear
2. Expand any acronyms and abbreviations in the original question by providing the full term. Include both the original abbreviated version and the expanded version in the rephrased question.
</rephrased_question>

Return a JSON object with the following fields:
   - 'technical_keywords': a list of relevant keywords from question
   - 'company_keywords': a list of company names
   - 'rephrased_question': the full rephrased question string


\n\nAssistant:The metadata for the user question {query}:

"""

#### Metadata Generation Using prompts and Claude LLm

In [9]:
# Helper Function to get the LLM response in desired format
def llm_ouput_to_json_time(llm_output):
    # Use regular expressions to find content between curly braces
    pattern = r"\{([^}]*)\}"
    matches = re.findall(pattern, llm_output)
    if len(matches) < 1:
        return "", []
    try:
        json_obj = json.loads("{" + matches[0] + "}")
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {str(e)}")
        return "", []
    return (json_obj["time_keyword_type"], json_obj["time_keywords"], json_obj["explanation"])

#### Time Keywords extraction

In [10]:
def time_keyword_extraction(prompt, question, bedrock_client): 
    prompt_format = prompt.format(
        query=question, most_recent_quarter="Q1'24", most_recent_year=2024)
    body = json.dumps({
        # "prompt": prompt_format,
        "anthropic_version": "bedrock-2023-05-31",
        "messages":[{
            "role": "user",
            "content": [{
                "type": "text",
                "text": prompt_format
            }]
        }],
        "top_k": 250,
        "top_p": 1,
        "max_tokens": 512,
        "temperature": 0,
        "stop_sequences": ["Human"]
    })
    response = bedrock_client.invoke_model(
        modelId = exp_config['generation_llm_model'],
        accept = 'application/json',
        contentType = 'application/json',
        body=body
    )
    response_body = json.loads(response.get('body').read())
    llm_output = response_body.get('content')[0]['text']

    time_keyword_type, time_kwds, explanation = llm_ouput_to_json_time(llm_output)
    input_token = int(response_body.get('usage')['input_tokens'])
    output_token = int(response_body.get('usage')['output_tokens'])
    return time_keyword_type, time_kwds, explanation, input_token, output_token
time_keyword_type, time_kwds, explanation, input_token, output_token = time_keyword_extraction(prompt=PROMPT_METADATA_GENERATION_time, question = question, bedrock_client=bedrock_client)
print(f"Time keywords generated: {time_keyword_type, time_kwds, explanation}")

print(f"Total Input Token: {input_token}")
print(f"Total Output Token: {output_token}")


Time keywords generated: ('specific_year', ['2022'], "The user is requesting the revenue for 3M in a specific year, which is 2022. Therefore, the time_keyword_type is 'specific_year' and the time_keyword is '2022'.")
Total Input Token: 906
Total Output Token: 83


In [11]:
# Helper Function to get the LLM response in desired format
def llm_output_kwd(llm_output):
    soup = BeautifulSoup(llm_output, "html.parser")
    keywords = soup.find("keywords".lower())

    keywords = re.sub("<[^<]+>", "", str(keywords))
    keywords = keywords.strip()
    if keywords:
        keywords = keywords.split(",")
        keywords = [keyword.strip() for keyword in keywords]

    return keywords

#### Technical Keywords extraction

In [12]:
def technical_keyword_extraction(prompt, question, bedrock_client):
    prompt_format = prompt.format(query=question)
    body = json.dumps({
        # "prompt": prompt_format,
        "anthropic_version": "bedrock-2023-05-31",
        "messages":[{
            "role": "user",
            "content": [{
                "type": "text",
                "text": prompt_format
            }]
        }],
        "top_k": 250,
        "top_p": 1,
        "max_tokens": 512,
        "temperature": 0,
        "stop_sequences": ["Human"]
    })
    response = bedrock_client.invoke_model(
        modelId = exp_config['generation_llm_model'],
        accept = 'application/json',
        contentType = 'application/json',
        body=body
    )
    response_body = json.loads(response.get('body').read())
    llm_output = response_body.get('content')[0]['text'] #('content')[0]['text'] or ('completion')
    input_token = int(response_body.get('usage')['input_tokens'])
    output_token = int(response_body.get('usage')['output_tokens'])
    kwds = llm_output_kwd(llm_output)
    return kwds, input_token, output_token 
kwds, input_token, output_token  = technical_keyword_extraction(prompt=PROMPT_METADATA_TECHNICAL_KWD, question = question, bedrock_client=bedrock_client)
print(kwds)

print(f"Total Input Token: {input_token}")
print(f"Total Output Token: {output_token}")

['revenue', 'sales', 'income', 'financial performance', 'financial results']
Total Input Token: 148
Total Output Token: 20


In [13]:
# Helper Functions to get the LLM response in desired format
def llm_ouput_to_json(llm_output):
    # Use regular expressions to find content between curly braces
    pattern = r"\{([^}]*)\}"
    matches = re.findall(pattern, llm_output)
    if len(matches) < 1:
        return "", []
    try:
        json_obj = json.loads("{" + matches[0] + "}")
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {str(e)}")
        return "", []
    return (
        json_obj["rephrased_question"],
        json_obj["technical_keywords"],
        json_obj["company_keywords"] if "company_keywords" in json_obj.keys() else [],
    )

def convert_quarter_format(input_string):
    # Ensure both quarter formats are included eg: Q2'22 <--> Q2 2022

    # Use regular expression to match "Q<quarter>'<year>"
    match = re.match(r"Q(\d)\'(\d{2})", input_string.strip())

    if match:
        quarter = match.group(1)
        year = "20" + match.group(2)
        return f"Q{quarter} {year}"

    match = re.match(r"Q(\d) (\d{4})", input_string.strip())

    if match:
        quarter = match.group(1)
        year = match.group(2)[-2:]
        return f"Q{quarter}'{year}"

    # For case of e.g., Q1 F2023
    match = re.match(r"Q(\d) F(\d{4})", input_string.strip())

    if match:
        quarter = match.group(1)
        year = match.group(2)[-2:]
        return f"Q{quarter}'{year}"

    # For case of e.g., F2023
    match = re.match(r"F(\d{4})", input_string.strip())

    if match:
        year = match.group(1)
        return f"{year}"

    return ""

#### Combined Metadata Generation

In [14]:
def metadata_extraction(prompt, question, bedrock_client, time_kwds):
    prompt_format = prompt.format(query=question, most_recent_quarter="Q1'24", time_kwds=time_kwds)
    body = json.dumps({
        # "prompt": prompt_format,
        "anthropic_version": "bedrock-2023-05-31",
        "messages":[{
            "role": "user",
            "content": [{
                "type": "text",
                "text": prompt_format
            }]
        }],
        "top_k": 250,
        "top_p": 1,
        "max_tokens": 512,
        "temperature": 0,
        "stop_sequences": ["Human"]
    })
    response = bedrock_client.invoke_model(
        modelId = exp_config['generation_llm_model'],
        accept = 'application/json',
        contentType = 'application/json',
        body=body
    )
    response_body = json.loads(response.get('body').read())
    llm_output = response_body.get('content')[0]['text']
    input_token = int(response_body.get('usage')['input_tokens'])
    output_token = int(response_body.get('usage')['output_tokens'])
    rephrased_q, kwds, company_keywords = llm_ouput_to_json(llm_output)
    time_kwds = time_kwds.split(",") if type(time_kwds) == str else time_kwds
    time_kwds = [time_kwd.strip() for time_kwd in time_kwds]

    new_time_kwds = []
    for time_kwd in time_kwds:
        new_time_kwd = convert_quarter_format(time_kwd)
        if new_time_kwd:
            new_time_kwds.append(new_time_kwd)
    time_kwds.extend(new_time_kwds)
    time_kwds = list(set(time_kwds))

    kwds = list(set(kwds))
    doc_type = []
    if len(time_kwds) == 0 or time_kwds == [""] or time_kwds == "none" or time_kwds is None:
        time_kwds = [2022,2023,2024]
    return company_keywords, time_kwds, kwds , rephrased_q, input_token, output_token 
company_keywords, time_kwds, kwds, rephrased_q, input_token, output_token  = metadata_extraction(prompt=PROMPT_METADATA_AND_QUERY_ReWr, question =question, bedrock_client =bedrock_client, time_kwds=time_kwds)
print(time_kwds)
print(company_keywords)
print(f"Total Input Token: {input_token}")
print(f"Total Output Token: {output_token}")

['2022']
['3M']
Total Input Token: 393
Total Output Token: 118


In [15]:
# Helper Functions to get the LLM response in desired format
def parse_time_kwds(time_kwds) -> list:
    """Given LLM generated time keywords, extract quarter and year

    Args:
        time_kwds (list): LLM generated time keywords

    Returns:
        list: List of tuples with year and quarter
    """
    set_tuple = set([])
    for kwd in time_kwds:
        match = re.match(r"Q(\d)\'(\d{2})", kwd.strip())
        if match:
            quarter = "q" + match.group(1)
            year = "20" + match.group(2)
            set_tuple.add((year, quarter))
            continue

        match = re.match(r"Q(\d) (\d{4})", kwd.strip())
        if match:
            quarter = "q" + match.group(1)
            year = match.group(2)
            set_tuple.add((year, quarter))
            continue

        match = re.match(r"Q(\d) F(\d{4})", kwd.strip())
        if match:
            quarter = "q" + match.group(1)
            year = match.group(2)
            set_tuple.add((year, quarter))
            continue

        match = re.search(r"(20\d{2})", kwd.strip())
        if match:
            quarter = ""
            year = match.group(1)
            set_tuple.add((year, quarter))
            continue

    return list(set_tuple)

def get_list_variables(company_kwds, time_kwds, kwds):
        time_kwds_tuples = parse_time_kwds(time_kwds)
        time_key_in_years = list(set([x[0] for x in time_kwds_tuples]))
        time_count = len(time_key_in_years)
        q_kwds = []

        return time_key_in_years, q_kwds

time_key_in_years, q_kwds = get_list_variables(company_keywords, time_kwds, kwds)

print(time_key_in_years)

['2022']


#### Chunks Retreival from Opensearch

In [16]:
def get_context(q, q_kwds, time_keyword_type, time_key_in_years, kwds, time_kwds, company_kwds):
    doc_type = []
    contexts = []
    retriever_embb = OpenSearchRetrieval(exp_config['opensearch_host'], exp_config['index_name_embb'])
    embedding = get_titan_text_embedding(q)
    contexts_sem = retriever_embb.retrieve_semantic(
        exp_config['emb_name'],
        embedding,
        time_kwds,
        company_kwds,
        doc_type,
        q_kwds,
        top_k=exp_config['top_k_retrieval'],
        use_company_kwds=True,
        use_doc_type=False,
    )

    retriever_text = OpenSearchRetrieval(exp_config['opensearch_host'], exp_config['index_name_embb'])
    contexts_text = retriever_text.retrieve_text(
        kwds,
        time_kwds,
        company_kwds,
        doc_type,
        exp_config['top_k_retrieval'],
        True,
        False,
    )
    contexts.extend(
        [
            context["paragraph"]
            for context in results_fusion([contexts_sem, contexts_text], [0.7, 0.3], top_k=exp_config['top_k_retrieval'])
        ]
    )

    return contexts

In [17]:
contexts = get_context( rephrased_q, q_kwds, time_keyword_type, time_key_in_years, kwds, time_kwds, company_keywords)

Using exact matching in semantic search with ['2022'], ['3M']
Using exact matching in text search with ['2022'], ['3M']


### Rerank the Retreived Chunks
##### Keyword based Reranker used for this Demo

In [18]:
def rerank_context(rephrased_query, kwds_list, time_kwds_list, company_kwds_list, contexts):
    # Set the top k ranking based on time keywords
    t0 = time.time()

    print("With keyword reranker")
    # Rank the contexts
    ranker = SearchRanker()
    combined_kwds = []
    combined_kwds.extend(set(kwds_list))
    combined_kwds.extend(set(time_kwds_list))
    combined_kwds.extend(set(company_kwds_list))
    ranked_contexts = ranker.rank_by_word_frequency(combined_kwds, contexts)
    
    top_k_contexts = ranked_contexts[: exp_config['top_k_ranking']]

    end_time = time.time()
    print(f"**** Time taken for reranking {end_time - t0}")
    return ranked_contexts, top_k_contexts

In [19]:
ranked_contexts, top_k_contexts = rerank_context(rephrased_q, q_kwds,time_kwds, company_keywords, contexts )

With keyword reranker
**** Time taken for reranking 0.0034983158111572266


In [20]:
#Check point to see if contexts/chunks are retreived
print (f"Number of Chunks retreived = {len(ranked_contexts)}")
print (f"Number of Ranked Chunks to be sent to llm = {len(top_k_contexts)}")

Number of Chunks retreived = 30
Number of Ranked Chunks to be sent to llm = 20


#### Prompt to generate Answer

In [21]:
PROMPT_ANS_GENERATION = """
\n\n
Human:
You a financial analyst that looks at a user question, related time and technical keywords and potentially relevant context.

<financial_information>
Paragraphs related to yearly and quarterly document reports: {context} \n
Financial Question related to yearly and Quarterly document Reports: {query} \n
Same financial question written in a different way: {rephrased_query} \n
Related Time Keywords: {time_kwds} \n
</financial_information>


<instruction>
To answer the question, think step by step:
1. Carefully read the question and any provided context paragraphs to find all the related paragraphs. Prioritize context paragraphs with csv tables.

2. If needed, analyze financial trends and quarter-over-quarter (Q/Q) performance over the detected time spans. Calculate rates of change between quarters to identify growth or decline.

3. Perform any required calculations to get the final answer, such as sums or divisions. Show the math.

4. Provide a complete, correct answer based on the given information. If information is missing, state what is needed to answer the question fully.

5. Present numerical values in rounded format using easy-to-read units.

6. Do not answer the question with "Based on the provided context" or anything similar. Just providing answer is enough.

7. Include the answer in a separate <answer></answer> tag with relevant and exhaustive information across all contexts. Substantiate your answer with explanation grounded in context. Conclude with a precise, concise, honest, and to-the-point answer.

8. Add the page source and number to <pages></pages> tag.

9. Add all Source Files from where the contexts were  used to generate the answers under a single <src></src> tag.
</instruction>


\n\n
Assistant:
"""


In [22]:
ans_gen_prompt = PROMPT_ANS_GENERATION.format(
            query=question,
            context=top_k_contexts,
            time_kwds=time_kwds,
            rephrased_query=rephrased_q,
            most_recent_quarter="Q1'24",
        )

### Answer Generation Using Claude LLM

In [23]:
def get_llm_answer(model_id: str, max_tokens: int = 800, temperature: float = 0.1, prompt: str = None):

    if model_id != "anthropic.claude-3-haiku-20240307-v1:0":
        body = json.dumps({
            "prompt": prompt,
            "top_k": 250,
            "top_p": 1,
            "max_tokens_to_sample": max_tokens,
            "temperature": 0,
            "stop_sequences": ["Question"]
        })
        response = bedrock_client.invoke_model(
            modelId = 'anthropic.claude-instant-v1',
            accept = 'application/json',
            contentType = 'application/json',
            body=body
        )
        response_body = json.loads(response.get('body').read())
        return response_body.get('completion')
    else:
        body = json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "messages":[{
                    "role": "user",
                    "content": [{
                        "type": "text",
                        "text": prompt
                    }]
                }],
                "top_k": 50,
                "top_p": 0.1,
                "max_tokens": max_tokens,
                "temperature": temperature,
                "stop_sequences": ["Human"],
            }
        )
        response = bedrock_client.invoke_model(
            modelId=model_id,
            body=body,
            accept = 'application/json',
            contentType = 'application/json'
        )

        response_body = json.loads(response.get('body').read())
        input_token = int(response_body.get('usage')['input_tokens'])
        output_token = int(response_body.get('usage')['output_tokens'])
        # print(response_body)
        # text
        return response_body.get('content')[0]['text'], input_token, output_token

In [24]:
# helper function to get the LLM response in desired format
def parse_generation(llm_prediction):
    soup = BeautifulSoup(llm_prediction, "html.parser")
    answer = soup.find("answer".lower())
    page_source = soup.find("pages".lower())
    source = soup.find("src".lower())

    answer = re.sub("<[^<]+>", "", str(answer))
    page_source = re.sub("<[^<]+>", "", str(page_source))
    source = re.sub("<[^<]+>", "", str(source))

    return answer+'\n'+page_source, source

In [25]:
prediction, input_token, output_token = get_llm_answer(model_id=exp_config['generation_llm_model'], max_tokens=4096, temperature=0.1, prompt=ans_gen_prompt)
answer, source = parse_generation(prediction)
print(answer)
print(source)
print(f"Total Input Token: {input_token}")
print(f"Total Output Token: {output_token}")


According to the financial information provided, 3M reported total net sales of $32,765 million in 2022.

The key evidence is from the following paragraphs:
27
"Net Sales: Refer to the preceding "Overview" section and the "Performance by Business Segment" section later in MD&amp;A for additional discussion of sales change."
19
"3M manages its operations in four operating business segments: Safety and Industrial; Transportation and Electronics; Health Care; and Consumer."

The total net sales figure of $32,765 million for 3M in 2022 is directly stated in the quarterly data table on 127.

27
3M_2022_10K
Total Input Token: 14825
Total Output Token: 179


### Bulk Processing - Proceed ONLY After Indexing is completed for all the files in notebook 01 

In [26]:
questions = ["What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.",
             "Is 3M a capital-intensive business based on FY2022 data?",
             "Does Adobe have an improving operating margin profile as of FY2022? If operating margin is not a useful metric for a company like this, then state that and explain why.",
             "Does Adobe have an improving Free cashflow conversion as of FY2022?",
             "Answer the following question as if you are an equity research analyst and have lost internet connection so you do not have access to financial metric providers. According to the details clearly outlined within the P&L statement and the statement of cash flows, what is the FY2015 depreciation and amortization (D&A from cash flow statement) % margin for AMD?",
             "From FY21 to FY22, excluding Embedded, in which AMD reporting segment did sales proportionally increase the most?",
             "How much has the effective tax rate of American Express changed between FY2021 and FY2022?",
             "What was the largest liability in American Express's Balance Sheet in 2022?",
             "What is the year end FY2019 total amount of inventories for Best Buy? Answer in USD millions. Base your judgments on the information provided primarily in the balance sheet.",
             "Are Best Buy's gross margins historically consistent (not fluctuating more than roughly 2% each year)? If gross margins are not a relevant metric for a company like this, then please state that and explain why."]

In [27]:
llm_answers =[]
llm_contexts =[]
latency_meta_time = []
latency_meta_kwd = []
latency_meta_comb = []
latency_meta_ans_gen = []
input_tokens = []
output_tokens = []
for question in questions:    
    t0=time.time()
    time_keyword_type, time_kwds, explanation, input_token1, output_token1 = time_keyword_extraction(
        prompt=PROMPT_METADATA_GENERATION_time,
        question = question,
        bedrock_client=bedrock_client)
    t1=time.time()
    kwds, input_token2, output_token2  = technical_keyword_extraction(
        prompt=PROMPT_METADATA_TECHNICAL_KWD,
        question = question,
        bedrock_client=bedrock_client)
    t2=time.time()
    company_keywords, time_kwds, kwds, rephrased_q, input_token3, output_token3 = metadata_extraction(
        prompt=PROMPT_METADATA_AND_QUERY_ReWr,
        question =question,
        bedrock_client =bedrock_client,
        time_kwds=time_kwds)
    t3=time.time()
    time_key_in_years, q_kwds = get_list_variables(
        company_keywords,
        time_kwds,
        kwds)
    contexts = get_context( rephrased_q,
                           q_kwds,
                           time_keyword_type,
                           time_key_in_years,
                           kwds,
                           time_kwds,
                           company_keywords)
    ranked_contexts, top_k_contexts = rerank_context(rephrased_q,
                                                     q_kwds,time_kwds,
                                                     company_keywords,
                                                     contexts )
    ans_gen_prompt = PROMPT_ANS_GENERATION.format(
                query=question,
                context=top_k_contexts,
                time_kwds=time_kwds,
                rephrased_query=rephrased_q,
                most_recent_quarter="Q1'24",
            )
    t4=time.time()
    prediction, input_token4, output_token4 = get_llm_answer(model_id=exp_config['generation_llm_model'],
                                max_tokens=4096,
                                temperature=0.1,
                                prompt=ans_gen_prompt)
    t5=time.time()
    answer, source = parse_generation(prediction)
    total_input_token = input_token1 + input_token2 + input_token3 + input_token4
    total_output_token = output_token1 + output_token2 + output_token3 + output_token4
    input_tokens = input_tokens + [total_input_token]
    output_tokens = output_tokens + [total_output_token]
    llm_answers = llm_answers + [answer.replace("\n", "")]
    llm_contexts = llm_contexts +[ranked_contexts]
    latency_meta_time = latency_meta_time + [t1-t0]
    latency_meta_kwd = latency_meta_kwd + [t2-t1]
    latency_meta_comb = latency_meta_comb + [t3-t2]
    latency_meta_ans_gen = latency_meta_ans_gen + [t5-t4]
    print(answer)
    print(source)
    print(f"****Total Time taken for Metadata extraction - time : {(t1-t0):.2f}")
    print(f"****Total Time taken for Metadata extraction - keywords : {(t2-t1):.2f}")
    print(f"****Total Time taken for Metadata extraction - combined : {(t3-t2):.2f}")
    print(f"****Total Time taken for Answer Generation : {(t5-t4):.2f}")


Using exact matching in semantic search with ['2018'], ['3M']
Using exact matching in text search with ['2018'], ['3M']
With keyword reranker
**** Time taken for reranking 0.008697748184204102

According to the cash flow statement in the 3M 2018 10-K report, the capital expenditure (purchases of property, plant and equipment) for fiscal year 2018 was $1,577 million.

Specifically, the cash flow statement shows the following:

"Purchases of property, plant and equipment (PP&amp;E)
2018: $1,577 million
2017: $1,373 million
2016: $1,420 million"

So the capital expenditure amount for 3M in fiscal year 2018 was $1,577 million.

46
3M_2018_10K
****Total Time taken for Metadata extraction - time : 0.93
****Total Time taken for Metadata extraction - keywords : 0.61
****Total Time taken for Metadata extraction - combined : 1.45
****Total Time taken for Answer Generation : 2.48
Using exact matching in semantic search with ['2022'], ['3M']
Using exact matching in text search with ['2022'], ['3M'

### Write to CSV for evaluation

In [28]:

# Open the input CSV file for reading
with open('../data/selected_samples.csv', 'r') as infile:
    reader = csv.DictReader(infile)
    # Open the output CSV file for writing
    fieldnames = reader.fieldnames  # Get the fieldnames from the input file
    
    # Add the new column names to the fieldnames list
    new_columns = ['latency_meta_time', 'latency_meta_kwd', 'latency_meta_comb', 'latency_meta_ans_gen', 'input_tokens', 'output_tokens']
    fieldnames.extend(new_columns)
    
    # # Open the output CSV file for writing
    # fieldnames = reader.fieldnames  # Get the fieldnames from the input file
    with open('../outputs/rag_outputs/output_claude.csv', 'w', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()  # Write the header row
        
        # Lists of new values to be updated in the specified columns
        llm_ans = llm_answers
        llm_contex = llm_contexts
        
        # Iterate over the rows in the input file and the new values
        for row_index, row in enumerate(reader, start=1):
            # Update the values in the desired columns for the current row
            if row_index <= len(llm_ans):
                row['llm_answer'] = llm_ans[row_index - 1]
            
            if row_index <= len(llm_contex):
                row['llm_contexts'] = llm_contex[row_index - 1]
                
            if row_index <= len(llm_contex):
                row['latency_meta_time'] = latency_meta_time[row_index - 1]
            
            if row_index <= len(llm_contex):
                row['latency_meta_kwd'] = latency_meta_kwd[row_index - 1]
            
            if row_index <= len(llm_contex):
                row['latency_meta_comb'] = latency_meta_comb[row_index - 1]
            
            if row_index <= len(llm_contex):
                row['latency_meta_ans_gen'] = latency_meta_ans_gen[row_index - 1]
            if row_index <= len(llm_contex):
                row['input_tokens'] = input_tokens[row_index - 1]
            
            if row_index <= len(llm_contex):
                row['output_tokens'] = output_tokens[row_index - 1]
            
            # Write the updated row to the output file
            writer.writerow(row)

### <center>----------EOF---------</center>